# Discrete event simulator

- Monte-Carlo modeling of car traffic in a city
- City is simulated as a grid of $N\times N$ rectilinear intersections
- One lane per direction.
- Cardinal directions only `[N, E, S, W]`
- Traffic lights at every intersection with fixed cycle
- Road segment between adjoint intersection, discretized

## Outline

- Discrete-event simulation (DES)

  - General idea: model a system where “interesting” changes happen at discrete times (car finishes a block, light turns green, etc.), not continuously.
  - Things to look up: event scheduling, state variables, performance measures (throughput, delay, queue length).

- Traffic flow as a queuing network

  - Each road segment is basically a server with limited capacity and a stochastic service time (your sample_link_travel_time).
  - The stop line at the signal is a FIFO queue with a saturation flow (how many cars per green tick can discharge).

- Signalized intersection control

  - The `signal_phase` function gives a fixed 2-phase controller: N/S is green for some number of ticks, then E/W.
  - This is the simplest form of fixed-time control (a.k.a. pre-timed signals). Search terms: cycle length, split, phase, saturation flow rate.

- Cellular automata traffic models (Nagel–Schreckenberg style)

  - A [1992 cellular automaton model](https://en.wikipedia.org/wiki/Nagel%E2%80%93Schreckenberg_model) of single-lane highways that inspired a ton of simple-but-realistic traffic simulations.

- Boundary conditions and injection rate

  - `is_boundary_incoming_link` and `ARRIVAL_RATE` together define demand loading: how cars enter the network. (If you want to go technical, this is the arrival rate in an M/M/1 queue, spatially distributed on the edges of the grid).


## Implementation preliminaries


In [ ]:
# Dependencies
from dataclasses import dataclass
import random
from collections import deque, defaultdict

# Size of grid
N = 8
# Length of simulation in time ticks
TOTAL_TICKS = 1_000

# Cardinal direction constants
NORTH = "N"
SOUTH = "S"
EAST = "E"
WEST = "W"

CLOCKWISE = [NORTH, EAST, SOUTH, WEST]

# Turn constants
STRAIGHT = "straight"
LEFT = "left"
RIGHT = "right"

# Turn probabilities [left, straight, right]
TURN_PROBABILITIES = [0.25, 0.50, 0.25]

# Traffic light timing
CYCLE_NS_GREEN = 20
CYCLE_EW_GREEN = 20
CYCLE_TOTAL = CYCLE_NS_GREEN + CYCLE_EW_GREEN

# How many cars car flow through a green light per time tick?
FLOW_PER_TICK = 1

# Link capacity: now many cars in transit are allowed between
# neighboring nodes
LINK_IN_TRANSIT_CAP = 50
# How many cars can wait at a stop light
QUEUE_CAP = 40

# Travel timing
BASE_TRAVEL_T = 6

## Microscoping modeling

An intersection is represented by a `Node` object. I use [`dataclass`](https://docs.python.org/3/library/dataclasses.html) to simplify implementation. A car is a simple object with a unique identifier and the time stamp for when entering the grid.


In [10]:
@dataclass(frozen=True)
class Node:
    i: int  # index of E-W street
    j: int  # index of N-S street


@dataclass
class Car:
    id: int  # unique identifier for the car
    t_enter: int  # time step when the car enters the grid

## Mesoscopic considerations

We must capture the direction of incoming and outgoing traffic to an intersection.


In [ ]:
def outgoing_for(node: Node) -> list[str]:
    """Returns list of (Node, direction) tuples for outgoing links from the node."""
    directions = []
    i, j = node.i, node.j  # for easy of use
    if i > 0:
        directions.append(
            (Node(i - 1, j), NORTH)
        )  # northbound traffic leaving to north
    if j < N - 1:
        directions.append((Node(i, j + 1), EAST))  # eastbound traffic leaving to east
    if i < N - 1:
        directions.append(
            (Node(i + 1, j), SOUTH)
        )  # southbound traffic leaving to south
    if j > 0:
        directions.append((Node(i - 1, j), WEST))  # westbound traffic leaving to west
    return directions


def incoming_for(node: Node) -> list[str]:
    """Returns list of (Node, direction) tuples for incoming links to the node."""
    directions = []
    i, j = node.i, node.j  # for easy of use
    if i < N - 1:
        directions.append(
            (Node(i + 1, j), NORTH)
        )  # northbound traffic arriving from south
    if j > 0:
        directions.append(
            (Node(i, j - 1), EAST)
        )  # eastbound traffic arriving from west
    if i > 0:
        directions.append(
            (Node(i - 1, j), SOUTH)
        )  # southbound traffic arriving from north
    if j < N - 1:
        directions.append(
            (Node(i, j + 1), WEST)
        )  # westbound traffic arriving from east
    return directions


def turn_direction(approach_direction: str) -> str:
    """Given the direction from which a car is approaching an intersection,
    return the direction the car will turn (left, straight, right)."""
    left, straight, right = TURN_PROBABILITIES
    # Flip a coin to determine turn
    rand = random.random()
    # Determine turn based on given probabilities
    if rand < left:
        turn = LEFT
    elif rand < straight:
        turn = STRAIGHT
    else:
        turn = RIGHT
    # Assume we are going straight
    new_direction = approach_direction
    if turn != STRAIGHT:
        # But if direction is not straight, find the index-value of
        # the current approach direction
        approach_idx = CLOCKWISE[approach_direction]
        # Assume we are turning left and update the index to counter-clockwise
        # direction by decrementing index and protecting wrap-around using modulo
        new_idx = (approach_idx - 1) % len(CLOCKWISE)
        if turn == RIGHT:
            # If we are turning right, update the index to clockwise direction
            new_idx = (approach_idx + 1) % len(CLOCKWISE)
        # Get the new direction string from the updated index
        new_direction = CLOCKWISE[new_idx]
    # Return the new direction string
    return new_direction


def is_boundary_incoming_link(src: Node, dst: Node) -> bool:
    """Links whose source is on the boundary and destination inside the grid as
    boundary arrival points."""
    si = src.i  # 0 is north boundary, N-1 south
    sj = src.j  # 0 is west boundary, N-1 east
    di = dst.i  # when == si traffic moves N/S, otherwise E/S
    dj = dst.j  # when == sj traffic moves E/W, otherwise N/N
    return (
        (si == 0 and di == si + 1 and sj == dj)  # from outside southbound
        or (si == N - 1 and di == si - 1 and sj == dj)  # from outside northbound
        or (sj == 0 and dj == sj + 1 and si == di)  # from outside eastbound
        or (sj == N - 1 and dj == sj - 1 and si == di)  # from outside westbound
    )


def signal_phase(t: int, node: Node) -> list:
    """Given the current time tick and node, return the list of directions that
    are green for the current signal phase."""
    tt = t % CYCLE_TOTAL
    # Assume east-west is green
    green_axis = [EAST, WEST]
    # Check if we are still in the north-south green phase
    if tt < CYCLE_NS_GREEN:
        # Update the green axis to north-south
        green_axis = [NORTH, SOUTH]
    # Return the direction in which the signal is green
    return green_axis


def add_travel_time(base=BASE_TRAVEL_T):
    """Place holder method so that we can introduce random jitter in
    addition to the base time."""
    return base

## Macroscopic considerations


In [ ]:
# For each directed link (u -> v) we have:
# - in_transit -- a queue of cars and their remaining time
# The downstream queue is stored as well as:
# - queues -- a queue of just cars lining up for one of the
# neighboring nodes.
in_transit = dict()  # key -> value: {(u,v) -> Car, remaining time}
queues = dict()  # key -> value: {(u,v) -> Car lining up at a neighbor light}

nodes = [Node(i, j) for i in range(N) for j in range(N)]
links = []
for node in nodes:
    for v, direction in outgoing_for(node):
        links.append((node, v))
        in_transit[(node, v)] = deque()
        queues[(node, v)] = deque()


def enqueue_departure(src: Node, dst: Node, car: Car) -> bool:
    """Put a car onto the link (src -> dst) in space allows."""
    # Let's see how many cars are currently on the link
    buf = in_transit[(src, dst)]
    success = len(buf) < LINK_IN_TRANSIT_CAP
    if success:
        # There is room; add the car with the default travel time
        buf.append((car, add_travel_time()))
    return success


def pop_to_queue_if_arrived(src: Node, dst: Node) -> int:
    """Cars ending their transit along the (src -> dst)
    link are moved into the downstream stop-light queue"""
    # Get all the cars along the src -> dst link
    buf = in_transit[((src, dst))]
    # Get the downstream queue
    q = queues[(src, dst)]
    # How many cars we move to the downstream stop-light queue?
    moved = 0
    # Decrease remaining time in the buffer
    for k in range(len(buf)):
        car, rt = buf[k]
        buf[k] = (car, rt - 1)
    # Any cars whose remaining time is <= 0, should be moved
    # to the downstream stop-light queue. To preserve their
    # order in the queue, we use popleft():
    tmp = deque()
    while buf:
        car, rt = buf.popleft()
        if rt <= 0:
            # Car is done traveling in this link. Move it to
            # the downstream queue if there is room
            if len(q) < QUEUE_CAP:
                q.append(car)
                moved += 1
            else:
                # Car is done traversing the link so set rt=0;
                # However there is no room for it at the stop-light
                # queue, so we have to hold it at the head of the
                # buffer for the next cycle.
                tmp.appendleft((car, 0))
        else:
            # Car still in transit, let's keep it in the buffer
            # with its remaining time in tact.
            tmp.append((car, rt))
        # update the link between src->dst
        in_transit[((src, dst))] = tmp
    # Done
    return moved


def serve_intersection(t: int, node: Node) -> int:
    """Clears queues for approaches that have a green light."""
    green_directions = signal_phase(t, node)
    served = 0  # number of cars moved through the light

    for u, approach_direction in incoming_for(node):

        # skip approaches that are not green
        if approach_direction in green_directions:
            # We are looking at the link from u -> node
            link_key = (u, node)
            # Get the queue for this incoming link
            q = queues[link_key]
            # Get the number of cars waiting at this light
            moves_attempted = 0
            stop_processing = False  # controls the while loop

            # up to FLOW_PER_TICK vehicles, but stop early if blocked or queue empties
            while moves_attempted < FLOW_PER_TICK and not stop_processing and q:
                car = q[0]

                outgoing_direction = turn_direction(approach_direction)

                # resolve destination node
                i, j = node.i, node.j
                next_node = None
                car_exits = False

                if outgoing_direction == NORTH and i > 0:
                    next_node = Node(i - 1, j)
                elif outgoing_direction == SOUTH and i < N - 1:
                    next_node = Node(i + 1, j)
                elif outgoing_direction == WEST and j > 0:
                    next_node = Node(i, j - 1)
                elif outgoing_direction == EAST and j < N - 1:
                    next_node = Node(i, j + 1)
                else:
                    # car leaves grid
                    car_exits = True

                if car_exits:
                    q.popleft()
                    record_completion(car, t)
                    served += 1
                    moves_attempted += 1
                else:
                    # try to enqueue onto the departing link. enqueue_departure
                    # returns true if successful
                    if enqueue_departure(node, next_node, car):
                        q.popleft()
                        served += 1
                        moves_attempted += 1
                    else:
                        # downstream link is full -> stop serving this approach this tick
                        stop_processing = True

    return served


def record_completion(car: Car, t_now: int) -> None:
    """Records the completion of a car's trip through the grid."""
    global completed, sum_tt
    completed += 1
    sum_tt = t_now - car.t_enter

## Simulator


In [ ]:
# Initialization of global statistics

car_id = 0
completed = 0
sum_tt = 0
queue_samples = 0
sum_queue = 0

For the simulator we need a loop that runs for a given length (determined by `TOTAL_TICKS`) to perform the following tasks during each iteration.

- Move along cars in links and push arrivals into downstream traffic-light queues.
- Serve each intersection according to its traffic light condition.
- Spawn boundary arrivals according to some random process.
- Update various metrics such as the total number of cars in traffic-light queues and the number of times we sample the queues.

After the simulator loop ends, we produce a report with the following data:

- Size of grid and duration of simulation
- Traffic light cycle information
- Arrival rate, at the boundary, per tick
- Total number of trips completed (from entering the grid to exiting)
- The throughput of the system defined as `completed/TOTAL_TICKS`
- The mean travel time in the grid defined as `sum_tt/completed`
- The mean number of queues vehicles defined as `sum_queue/max(1, queue_samples)`
